In [3]:
# 1. Imports
import sys
sys.path.append('./sportmonks_api')

import pandas as pd
from sportmonks_calls import Fixtures

# 2. API Key
API_KEY = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"

# 3. Load saved seasons
seasons_df = pd.read_csv("seasons.csv")
print(f"Loaded {len(seasons_df)} seasons.")

# 4. Initialize Fixtures API
fixtures_api = Fixtures(API_KEY)

# 5. Fetch fixtures for each season
all_fixtures = []

for idx, row in seasons_df.iterrows():
    season_id = row['season_id']
    league_name = row['league_name']

    print(f"Fetching fixtures for Season ID {season_id} ({league_name})...")

    # ✅ Correct v3 URL
    endpoint = f"https://api.sportmonks.com/v3/football/fixtures?seasons={season_id}"

    # Fetch fixtures
    success = fixtures_api.request.make_request(endpoint, paginated=True)

    if not success:
        print(f"Failed to fetch fixtures for season {season_id}. Skipping...")
        continue

    fixtures = fixtures_api.request.results
    for fixture in fixtures:
        all_fixtures.append(fixture)

# 6. Build dataframe
fixtures_df = pd.DataFrame(all_fixtures)
print(f"Total fixtures fetched: {len(fixtures_df)}")

# 7. (Simple view)
display(fixtures_df.head())

# 8. (Optionally save later)
# fixtures_df.to_csv("fixtures.csv", index=False)


Loaded 531 seasons.
Fetching fixtures for Season ID 2 (Premier League)...


KeyboardInterrupt: 

In [4]:
import pandas as pd
from sportmonks_calls import Fixtures
import os

# 1. Load seasons
seasons_df = pd.read_csv("seasons.csv")
print(f"Loaded {len(seasons_df)} seasons.")

# 2. Initialize Fixtures API
API_KEY = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"
fixtures_api = Fixtures(API_KEY)

# 3. Prepare fetching
season_ids = list(seasons_df['season_id'])
batch_size = 5  # Can later change to 10 if stable

# 4. Check if partial backup exists
if os.path.exists("fixtures_partial.csv"):
    fixtures_df = pd.read_csv("fixtures_partial.csv")
    fetched_ids = set(fixtures_df['season_id'].unique())
    print(f"Resuming from {len(fetched_ids)} already fetched seasons.")
else:
    fixtures_df = pd.DataFrame()
    fetched_ids = set()

all_fixtures = []

# 5. Loop over batches
for i in range(0, len(season_ids), batch_size):
    batch = season_ids[i:i+batch_size]

    # Skip batch if all seasons already fetched
    if all(season_id in fetched_ids for season_id in batch):
        continue

    batch_str = ",".join(map(str, batch))
    print(f"Fetching fixtures for seasons: {batch_str}")

    # Build endpoint manually
    endpoint = f"https://api.sportmonks.com/v3/football/fixtures?seasons={batch_str}"

    success = fixtures_api.request.make_request(endpoint, paginated=True)

    if not success:
        print(f"Failed to fetch fixtures for seasons: {batch_str}")
        continue

    fixtures = fixtures_api.request.results
    print(f"Fetched {len(fixtures)} fixtures.")

    # Add batch results
    batch_df = pd.DataFrame(fixtures)
    batch_df['season_id'] = batch  # Add season info
    fixtures_df = pd.concat([fixtures_df, batch_df], ignore_index=True)

    # Save partial progress every batch
    fixtures_df.to_csv("fixtures_partial.csv", index=False)
    print(f"Checkpoint saved after batch {batch_str}!")

# 6. Final save
fixtures_df.to_csv("fixtures.csv", index=False)
print(f"✅ Saved all {len(fixtures_df)} fixtures to fixtures.csv!")
print("You can safely delete fixtures_partial.csv if you want after verifying.")


Loaded 531 seasons.
Fetching fixtures for seasons: 2,3,6,7,8


KeyboardInterrupt: 

In [5]:
# 03_fetch_fixtures.ipynb

import requests
import pandas as pd
import time

# 1. API Key
API_KEY = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"

# 2. Base URL
BASE_URL = "https://api.sportmonks.com/v3/football/fixtures"

# 3. Initialize
all_fixtures = []
current_page = 1
save_every_x_pages = 20  # <- Save every 20 pages
output_file = "fixtures_raw.csv"

# 4. Fetch loop
while True:
    print(f"Fetching page {current_page}...")
    
    params = {
        "api_token": API_KEY,
        "page": current_page,
    }
    
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code} {response.text}")
        break
    
    data = response.json()
    
    if 'data' not in data:
        print(f"No more data on page {current_page}. Exiting loop.")
        break

    fixtures = data['data']
    all_fixtures.extend(fixtures)
    
    # Save every X pages to be safe
    if current_page % save_every_x_pages == 0:
        print(f"Saving partial results after page {current_page}...")
        pd.DataFrame(all_fixtures).to_csv(output_file, index=False)
    
    # Pagination
    if 'pagination' in data and data['pagination']['has_more']:
        current_page += 1
    else:
        print("No more pages left.")
        break

    # Sleep optional (commented for now)
    # time.sleep(0.2)

# 5. Final save
print(f"Saving final results with {len(all_fixtures)} fixtures...")
pd.DataFrame(all_fixtures).to_csv(output_file, index=False)

print("✅ Done fetching all fixtures!")


Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Fetching page 19...
Fetching page 20...
Saving partial results after page 20...
Fetching page 21...
Fetching page 22...
Fetching page 23...
Fetching page 24...
Fetching page 25...
Fetching page 26...
Fetching page 27...
Fetching page 28...
Fetching page 29...
Fetching page 30...
Fetching page 31...
Fetching page 32...
Fetching page 33...
Fetching page 34...
Fetching page 35...
Fetching page 36...
Fetching page 37...
Fetching page 38...
Fetching page 39...
Fetching page 40...
Saving partial results after page 40...
Fetching page 41...
Fetching page 42...
Fetching page 43...
Fetching page 44...
Fetching page 45...
Fetching page 46...
Fetching 

KeyboardInterrupt: 

In [7]:
import requests
import pandas as pd
from tqdm import tqdm
import time

# 1. Set API details
API_KEY = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"
BASE_URL = "https://api.sportmonks.com/v3/football/fixtures"

# 2. Output file
output_file = "fixtures_full.csv"
save_every_x_pages = 25  # save every 25 pages

# 3. Initialize
all_fixtures = []
current_page = 1

# 4. Test connection
print("Testing API connection...")
first_response = requests.get(BASE_URL, params={"api_token": API_KEY, "page": 1})

if first_response.status_code != 200:
    print(f"Error: {first_response.status_code} {first_response.text}")
    raise Exception("Can't connect to Sportmonks API.")

first_data = first_response.json()

# Set estimated progress for tqdm
estimated_total = 1000
print("Connection OK. Starting fetching without fixed total pages...")

# 5. Fetch loop with progress bar
pbar = tqdm(total=estimated_total, desc="Fetching Fixtures", position=0, leave=True)

while True:
    try:
        params = {
            "api_token": API_KEY,
            "page": current_page,
        }

        response = requests.get(BASE_URL, params=params, timeout=15)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code} {response.text}")
            break

        data = response.json()

        if 'data' not in data:
            print(f"No 'data' field on page {current_page}. Exiting loop.")
            break

        fixtures = data['data']
        all_fixtures.extend(fixtures)

        if current_page % save_every_x_pages == 0:
            pd.DataFrame(all_fixtures).to_csv(output_file, index=False)
            print(f"[Auto-Save] Saved {len(all_fixtures)} fixtures so far...")

        pbar.update(1)

        if 'pagination' in data and data['pagination']['has_more']:
            current_page += 1
            time.sleep(0.5)  # polite sleep to avoid hitting limits
        else:
            print("No more pages.")
            break

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}. Retrying in 5 seconds...")
        time.sleep(5)
        continue

pbar.close()

# Final save
pd.DataFrame(all_fixtures).to_csv(output_file, index=False)
print(f"All done! Total fixtures saved: {len(all_fixtures)} into {output_file}")

Testing API connection...
Connection OK. Starting fetching without fixed total pages...


Fetching Fixtures:   2%|▎         | 25/1000 [00:15<10:27,  1.55it/s]

[Auto-Save] Saved 625 fixtures so far...


Fetching Fixtures:   5%|▌         | 50/1000 [00:31<10:17,  1.54it/s]

[Auto-Save] Saved 1250 fixtures so far...


...

[Auto-Save] Saved 154375 fixtures so far...


Fetching Fixtures: 6200it [1:08:12,  1.23it/s]

[Auto-Save] Saved 155000 fixtures so far...


Fetching Fixtures: 6222it [1:08:27,  1.51it/s]


No more pages.
All done! Total fixtures saved: 155541 into fixtures_full.csv


In [8]:
import pandas as pd

# Assuming `all_fixtures` is the list containing your fetched fixture data
df_fixtures = pd.DataFrame(all_fixtures)
df_fixtures.to_csv("fixtures_full.csv", index=False)
print("Saved fixtures to fixtures_full.csv")


Saved fixtures to fixtures_full.csv


In [11]:
import json

# 1. Read the fixtures file
fixtures_df = pd.read_csv("fixtures_full.csv", low_memory=False)
fixture_ids = fixtures_df["id"].dropna().astype(int).tolist()

# 2. API and output
stats_output_file = "fixture_stats.csv"
save_every_x = 500  # save every 500 stats
stats_data = []
includes = "stats,localTeam,visitorTeam"

print(f"Starting to fetch statistics for {len(fixture_ids)} fixtures...")

# 3. Progress loop
for idx, fixture_id in enumerate(tqdm(fixture_ids, desc="Fetching Statistics")):
    try:
        url = f"https://api.sportmonks.com/v3/football/fixtures/{fixture_id}"
        params = {
            "api_token": API_KEY,
            "include": includes
        }
        response = requests.get(url, params=params, timeout=10)

        if response.status_code == 404:
            continue
        elif response.status_code != 200:
            print(f"[Error] Fixture {fixture_id}: {response.status_code}")
            continue

        data = response.json()
        if "data" not in data or "stats" not in data["data"]:
            continue

        fixture_stats = data["data"]
        fixture_stats["fixture_id"] = fixture_id  # Ensure ID is preserved
        stats_data.append(fixture_stats)

        if (idx + 1) % save_every_x == 0:
            pd.DataFrame(stats_data).to_json(stats_output_file, orient="records", lines=True)
            print(f"[Auto-Save] {len(stats_data)} stats saved to {stats_output_file}...")

        time.sleep(0.25)  # avoid rate limit

    except Exception as e:
        print(f"[Exception] Fixture {fixture_id}: {e}")
        time.sleep(5)
        continue

# 4. Final save
pd.DataFrame(stats_data).to_json(stats_output_file, orient="records", lines=True)
print(f"✅ Done! Total stats saved: {len(stats_data)} into {stats_output_file}")

Starting to fetch statistics for 155541 fixtures...


[Exception] Fixture 92833: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


[Exception] Fixture 93202: HTTPSConnectionPool(host='api.sportmonks.com', port=443): Read timed out. (read timeout=10)


[Exception] Fixture 16818824: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


[Exception] Fixture 16819790: HTTPSConnectionPool(host='api.sportmonks.com', port=443): Read timed out. (read timeout=10)


...

[Exception] Fixture 19131355: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


[Exception] Fixture 19134623: HTTPSConnectionPool(host='api.sportmonks.com', port=443): Read timed out. (read timeout=10)


[Exception] Fixture 19135361: HTTPSConnectionPool(host='api.sportmonks.com', port=443): Read timed out. (read timeout=10)


Fetching Statistics: 100%|██████████| 155541/155541 [11:51:55<00:00,  3.64it/s]

✅ Done! Total stats saved: 0 into fixture_stats.csv
